In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(100_000_000_000)

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca

import shared_utils

from siuba import *
import pandas as pd
import geopandas as gpd
import shapely

import datetime as dt
import time
from zoneinfo import ZoneInfo

import rt_analysis as rt
import importlib

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
bbb_itp_id = 300

In [3]:
analysis_date = dt.date(2022, 3, 30)

In [4]:
pbar = tqdm()

0it [00:00, ?it/s]

In [5]:
bbb = rt.OperatorDayAnalysis(bbb_itp_id, analysis_date, pbar)

found parquet
found parquet
found parquet
found parquet
found parquet
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less t

### Proof of Concept

  * A rough interative variability chart and map
    * Could be iterated on but would need to have a way to run a remote kernel in analyses site...

In [6]:
import rt_filter_map_plot

In [7]:
bbb2 = rt_filter_map_plot.from_gcs(bbb_itp_id, analysis_date)

found parquet


In [8]:
bbb2.set_filter(start_time='15:00', end_time='19:00')

In [9]:
from ipywidgets import interact, fixed
import ipywidgets as widgets

In [22]:
min_stop_seq = bbb2.stop_delay_view.stop_sequence.min()
max_stop_seq = bbb2.stop_delay_view.stop_sequence.max()
min_stop = widgets.IntSlider(min = min_stop_seq, max = max_stop_seq, value = min_stop_seq)
max_stop = widgets.IntSlider(min = min_stop_seq, max = max_stop_seq, value = max_stop_seq)

In [23]:
def friendly_chart_variability(rt_filter_mapper, min_stop_seq, max_stop_seq):
    return rt_filter_mapper.chart_variability(min_stop_seq = min_stop_seq, max_stop_seq = max_stop_seq)

In [24]:
bbb2.set_filter(start_time='15:00', end_time='19:00', shape_ids=['25951'])

In [27]:
interact(friendly_chart_variability, rt_filter_mapper = fixed(bbb2),
         min_stop_seq = min_stop, max_stop_seq = max_stop)

interactive(children=(IntSlider(value=9, description='min_stop_seq', max=56, min=1), IntSlider(value=37, descr…

<function __main__.friendly_chart_variability(rt_filter_mapper, min_stop_seq, max_stop_seq)>

In [14]:
bbb2_routes = list(bbb2.rt_trips.route_short_name.unique())
all_routes = widgets.SelectMultiple(
    options=bbb2_routes,
    value=bbb2_routes[:1],
    #rows=10,
    description='Routes',
    disabled=False
)

In [15]:
def filter_then_map(filter_mapper, start_time, end_time, route_names, shape_ids, direction_id, direction):
    filter_mapper.set_filter(start_time = start_time, end_time = end_time,
        route_names = all_routes.value, shape_ids = shape_ids,
        direction_id = direction_id, direction = direction)
    return filter_mapper.segment_speed_map(how='low_speeds')

In [16]:
interact(filter_then_map, filter_mapper = fixed(bbb2), start_time = fixed('15:00'), end_time = fixed('19:00'),
        route_names = all_routes, shape_ids = fixed(False),
        direction_id = fixed(False), direction = fixed(False))

interactive(children=(SelectMultiple(description='Routes', index=(0,), options=('41', '1', '7', '3', '2', '17'…

<function __main__.filter_then_map(filter_mapper, start_time, end_time, route_names, shape_ids, direction_id, direction)>